# Algorithm 1: Billing Model for Retail Markets

## Gather data to one table

In [1]:
import pandas as pd
import numpy as np
import random

profile = pd.read_excel("ele profile.xlsx").T


pro = profile.sample(frac=0.2)
con = profile[~profile.index.isin(pro.index)]

con=con.T

PV = pd.read_excel("PV profile.xlsx").T
WT = pd.read_excel("WT profile.xlsx").T

WT1=WT.iloc[5:10].T
WT2 = WT.iloc[0:5].T
WT_PV_CON = pro.iloc[20:25].T
PV1 = PV.iloc[20:25].T

PV_CON = pro.iloc[0:20].T
PV2 = PV.iloc[0:20].T

only_WT_CON = pro.iloc[25:30].T

# hour = 10

all_list=[]


for hour in range(168):
    statistics = []
    i=5
    
    for j in list(only_WT_CON.columns):
            statistics.append([j,
                only_WT_CON[j][hour],
                0,
                WT1[i][hour],
                WT1[i][hour]-only_WT_CON[j][hour]])
            i+=1
    i=0
    j=20
    z=0
    
    for i in list(WT_PV_CON.columns):

            statistics.append([i,
                WT_PV_CON[i][hour],
                PV1[j][hour] ,
                WT2[z][hour],
                WT2[z][hour]+PV1[j][hour]-WT_PV_CON[i][hour]])
            j+=1
            z+=1
    j=0
    
    for i in list(PV_CON.columns):

        statistics.append([i,
            PV_CON[i][hour],
            PV2[j][hour],
            0,
            PV2[j][hour]-PV_CON[i][hour]])
        j+=1
        
    for i in list(con.columns):
        statistics.append([i,
            con[i][hour],
            0,
            0,
            -con[i][hour]])

    a=pd.DataFrame(statistics, columns=['Client ID','Consumption data','PV data','WT','Consumption/generation profile']
               ,index=[x[0] for x in statistics])
    
    buyer = []
    seller = []
    for i in range(150):
        if statistics[i][4] < 0:
            statistics[i][4] = -statistics[i][4]
            statistics[i].append('buyer')
            buyer.append(statistics[i][0])
        else:
            statistics[i].append('seller')
            seller.append(statistics[i][0])


    b=list(a.index)
    random.shuffle(b)
    perfect = b[0:15]
    good = b[15:105]
    ok = b[105:135]
    bad=b[135:150]

    for i in range(150):

        if i in list(perfect):
            statistics[i].append('perfect')
            statistics[i].append(statistics[i][4])
        if i in list(good):
            statistics[i].append('good')
            statistics[i].append(random.choice([round(statistics[i][4]*random.uniform(0.9,0.99),2),
                                 round(statistics[i][4]*random.uniform(1.01,1.1),2)]))
        if i in list(ok):
            statistics[i].append('ok')
            statistics[i].append(random.choice([round(statistics[i][4]*random.uniform(0.8,0.89),2),
                                 round(statistics[i][4]*random.uniform(1.11,1.2),2)]))
        if i in list(bad):
            statistics[i].append('bad')
            statistics[i].append(random.choice([round(statistics[i][4]*random.uniform(0,0.79),2),
                                 round(statistics[i][4]*random.uniform(1.21,2),2)]))


    buyerl = []
    sellerl = []
    c1 = 0
    c2 = 0

    sorted_stat = []
    sorted_stat = sorted(statistics, key=(lambda x: x[7]))

    for i in range(len(seller)):
        sellerl.append(round(random.uniform(0.06, 0.2), 2))
    for j in range(len(buyer)):
        buyerl.append(round(random.uniform(0.05, 0.19), 2))

    sellerl = sorted(sellerl, reverse=True)
    buyerl = sorted(buyerl)

    for i in range(len(sorted_stat)):
        if sorted_stat[i][5] == 'seller':
            sorted_stat[i].append(sellerl[c1])
            c1 += 1
        else:
            sorted_stat[i].append(buyerl[c2])
            c2 += 1

    statistics = sorted_stat

    now = buyerl[0]
    buyery = []
    buyercount = []
    n = 0
    for i in range(len(buyerl)):
        if now == buyerl[i]:
            n += 1
        else:

            buyercount.append(n)
            buyery.append(now)
            now = buyerl[i]
            n = 1

    buyery.append(now)
    buyery = list(reversed(buyery))

    buyerx = []
    co = 0
    for i in buyery:

        for j in range(len(statistics)):
            if statistics[j][5] == 'buyer':
                if statistics[j][8] == i:
                    co += statistics[j][7]

        buyerx.append(co)

    now = sellerl[0]
    sellery = []
    sellercount = []
    n = 0
    for i in range(len(sellerl)):
        if now == sellerl[i]:
            n += 1
        else:

            sellercount.append(n)
            sellery.append(now)
            now = sellerl[i]
            n = 1

    sellery.append(now)

    sellery = list(reversed(sellery))
    sellerx = []
    co = 0
    for i in sellery:

        for j in range(len(statistics)):
            if statistics[j][5] == 'seller':
                if statistics[j][8] == i:
                    co += statistics[j][7]

        sellerx.append(co)

    c = buyery[2]     #intersecation

    for i in range(len(statistics)):
        if statistics[i][5] == 'buyer':
            if statistics[i][8] > c:
                statistics[i].append('accepted')
            else:
                statistics[i].append('rejected')
        else:
            if statistics[i][8] < c:
                statistics[i].append('accepted')
            else:
                statistics[i].append('rejected')


    statistics.sort()

    all_list.append(statistics)


## Assign supplier to each household

In [2]:
random.shuffle(all_list[len(all_list)-1])

for i in range(45):
    all_list[len(all_list)-1][i].append("supplier1")
    all_list[len(all_list)-1][i].append(0.2)

for i in range(45, 90):
    all_list[len(all_list)-1][i].append("supplier2")
    all_list[len(all_list)-1][i].append(0.205)

for i in range(90, 120):
    all_list[len(all_list)-1][i].append("supplier3")
    all_list[len(all_list)-1][i].append(0.21)

for i in range(120, 135):
    all_list[len(all_list)-1][i].append("supplier4")
    all_list[len(all_list)-1][i].append(0.215)

for i in range(135, 150):
    all_list[len(all_list)-1][i].append("supplier5")
    all_list[len(all_list)-1][i].append(0.22)


all_list[len(all_list)-1].sort()


for i in range(len(all_list)-1):
    for j in range(len(all_list[i])):
        all_list[i][j].append(all_list[len(all_list)-1][j][10])
        all_list[i][j].append(all_list[len(all_list)-1][j][11])

In [3]:
stat=all_list[1]
a = pd.DataFrame(stat, columns=['Client ID',
                                      'Consumption data',
                                      'PV data',
                                      'WT',
                                      'Consumption/generation profile',
                                      'Role', 
                                      'Prediction algorithm',
                                      'Predicted volume',
                                      'Price offered',
                                      'Bid',
                                      'supplier',
                                      'retail price']
                 , index=[x[0] for x in stat])
a.T.sort_index(axis=1)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
Client ID,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
Consumption data,0.05,0.05,0.07,0.2,0.14,0.08,0.09,0.16,0.05,0.11,...,0.03,0.05,0.2,0.03,0.1,0.16,0.06,0.16,0.15,0.11
PV data,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Consumption/generation profile,0.05,0.05,0.07,0.2,0.14,0.08,0.09,0.16,0.05,0.11,...,0.03,0.05,0.2,0.03,0.1,0.16,0.06,0.16,0.15,0.11
Role,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,...,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer
Prediction algorithm,good,ok,good,good,good,good,bad,ok,ok,good,...,good,ok,good,good,good,ok,good,good,ok,good
Predicted volume,0.05,0.06,0.07,0.19,0.13,0.07,0.13,0.18,0.04,0.12,...,0.03,0.06,0.2,0.03,0.09,0.18,0.06,0.17,0.13,0.1
Price offered,0.07,0.08,0.09,0.19,0.17,0.09,0.17,0.18,0.06,0.16,...,0.06,0.09,0.19,0.06,0.14,0.19,0.09,0.18,0.17,0.15
Bid,rejected,rejected,rejected,accepted,rejected,rejected,rejected,accepted,rejected,rejected,...,rejected,rejected,accepted,rejected,rejected,accepted,rejected,accepted,rejected,rejected


In [4]:
#alg1


supplier_bal=[]

alg1 = all_list
for i in range(len(alg1)):
    supplier_in1,supplier_in2,supplier_in3,supplier_in4,supplier_in5=0,0,0,0,0
    supplier_out1,supplier_out2,supplier_out3,supplier_out4,supplier_out5=0,0,0,0,0
    supplier_bal1,supplier_bal2,supplier_bal3,supplier_bal4,supplier_bal5=0,0,0,0,0

    for j in range(len(alg1[i])):
        Consumer_Bill=0
        Pro_Bill=0

        if alg1[i][j][5] == 'buyer':
            Consumer_Bill = alg1[i][j][4]*alg1[i][j][11]
        else:
            Pro_Bill = alg1[i][j][4]*0.05

        if alg1[i][j][10] == 'supplier1':
            alg1[i][j].append(Consumer_Bill)
            alg1[i][j].append(Pro_Bill)
            supplier_in1 = Consumer_Bill
            supplier_out1 = Pro_Bill
            supplier_bal1 += supplier_in1 - supplier_out1
        if alg1[i][j][10] == 'supplier2':
            alg1[i][j].append(Consumer_Bill)
            alg1[i][j].append(Pro_Bill)
            supplier_in2 = Consumer_Bill
            supplier_out2 = Pro_Bill
            supplier_bal2 += supplier_in2 - supplier_out2
        if alg1[i][j][10] == 'supplier3':
            alg1[i][j].append(Consumer_Bill)
            alg1[i][j].append(Pro_Bill)
            supplier_in3 = Consumer_Bill
            supplier_out3 = Pro_Bill
            supplier_bal3 += supplier_in3 - supplier_out3
        if alg1[i][j][10] == 'supplier4':
            alg1[i][j].append(Consumer_Bill)
            alg1[i][j].append(Pro_Bill)
            supplier_in4 = Consumer_Bill
            supplier_out4 = Pro_Bill
            supplier_bal4 += supplier_in4 - supplier_out4
        if alg1[i][j][10] == 'supplier5':
            alg1[i][j].append(Consumer_Bill)
            alg1[i][j].append(Pro_Bill)
            supplier_in5 = Consumer_Bill
            supplier_out5 = Pro_Bill
            supplier_bal5 += supplier_in5 - supplier_out5
    supplier_bal.append([supplier_bal1,supplier_bal2,supplier_bal3,
                               supplier_bal4,supplier_bal5])

In [5]:
stat=alg1[0]
a = pd.DataFrame(stat, columns=['Client ID',
                                      'Consumption data',
                                      'PV data',
                                      'WT',
                                      'Consumption/generation profile',
                                      'Role',
                                      'Prediction algorithm',
                                      'Predicted volume',
                                      'Price offered',
                                      'Bid',
                                      'supplier',
                                      'retail price',
                                'consumer bills', 'prosumer rewards']
                 , index=[x[0] for x in stat])
a.T.sort_index(axis=1)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
Client ID,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
Consumption data,0.05,0.05,0.06,0.09,0.09,0.09,0.09,0.14,0.05,0.1,...,0.02,0.05,0.09,0.04,0.11,0.09,0.06,0.11,0.14,0.11
PV data,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Consumption/generation profile,0.05,0.05,0.06,0.09,0.09,0.09,0.09,0.14,0.05,0.1,...,0.02,0.05,0.09,0.04,0.11,0.09,0.06,0.11,0.14,0.11
Role,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,...,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer
Prediction algorithm,good,good,ok,good,good,good,bad,ok,bad,good,...,good,good,good,good,good,good,ok,good,bad,good
Predicted volume,0.05,0.05,0.07,0.1,0.1,0.09,0.14,0.12,0.09,0.11,...,0.02,0.05,0.08,0.04,0.11,0.09,0.05,0.1,0.24,0.12
Price offered,0.08,0.08,0.1,0.13,0.13,0.12,0.17,0.15,0.11,0.14,...,0.05,0.08,0.11,0.07,0.15,0.13,0.08,0.13,0.19,0.16
Bid,rejected,rejected,rejected,rejected,rejected,rejected,rejected,rejected,rejected,rejected,...,rejected,rejected,rejected,rejected,rejected,rejected,rejected,rejected,accepted,rejected


# Algorithm 2: Billing Model with Individual Cost Split

In [6]:
#alg2
alg2 = []
supplier_bal_alg2 = []
alg2 = all_list
for i in range(len(alg2)):
    supplier_in1,supplier_in2,supplier_in3,supplier_in4,supplier_in5=0,0,0,0,0
    supplier_out1,supplier_out2,supplier_out3,supplier_out4,supplier_out5=0,0,0,0,0
    supplier_bal1,supplier_bal2,supplier_bal3,supplier_bal4,supplier_bal5=0,0,0,0,0
    supplier_out,supplier_in=0,0



    for j in range(len(alg2[i])):
        P2P_Bill=0
        P2P_Reward=0
        supplier_out,supplier_in=0,0
        supplier_in1,supplier_in2,supplier_in3,supplier_in4,supplier_in5=0,0,0,0,0
        supplier_out1,supplier_out2,supplier_out3,supplier_out4,supplier_out5=0,0,0,0,0
        if alg2[i][j][9] == 'accepted':
            supplier_out,supplier_in=0,0


            if alg2[i][j][4]-alg2[i][j][7] == 0:
               if alg2[i][j][5] == 'buyer':
                   P2P_Bill = alg2[i][j][7] * alg2[i][j][8]
               else:
                   P2P_Reward = alg2[i][j][7] * alg2[i][j][8]

               supplier_out,supplier_in=0,0

            if alg2[i][j][4]-alg2[i][j][7] < 0:
                if alg2[i][j][5] == 'buyer':
                    P2P_Bill = alg2[i][j][7] * alg2[i][j][8] + (alg2[i][j][4]-alg2[i][j][7])*0.05
                    supplier_out = -(alg2[i][j][4]-alg2[i][j][7])*0.05
                else:
                    P2P_Reward = alg2[i][j][7] * alg2[i][j][8] + (alg2[i][j][4]-alg2[i][j][7])*alg2[i][j][11]
                    supplier_in = -(alg2[i][j][4]-alg2[i][j][7])*alg2[i][j][11]

            if alg2[i][j][4]-alg2[i][j][7] > 0:
                if alg2[i][j][5] == 'buyer':
                   P2P_Bill = alg2[i][j][7] * alg2[i][j][8] + (alg2[i][j][4]-alg2[i][j][7])*alg2[i][j][11]
                   supplier_in = (alg2[i][j][4]-alg2[i][j][7])*alg2[i][j][11]
                else:
                    P2P_Reward = alg2[i][j][7] * alg2[i][j][8] + (alg2[i][j][4]-alg2[i][j][7])*0.05
                    supplier_out = (alg2[i][j][4]-alg2[i][j][7])*0.05

            alg2[i][j].append(P2P_Bill)
            alg2[i][j].append(P2P_Reward)

            if alg2[i][j][10] == 'supplier1':

                supplier_bal1 += supplier_in - supplier_out
            if alg2[i][j][10] == 'supplier2':

                supplier_bal2 += supplier_in - supplier_out
            if alg2[i][j][10] == 'supplier3':

                supplier_bal3 += supplier_in - supplier_out
            if alg2[i][j][10] == 'supplier4':

                supplier_bal4 += supplier_in - supplier_out
            if alg2[i][j][10] == 'supplier5':

                supplier_bal5 += supplier_in - supplier_out

        if alg2[i][j][9] == 'rejected':

            if alg2[i][j][5] == 'buyer':
                P2P_Bill = alg2[i][j][4]*alg2[i][j][11]
            else:
                P2P_Reward = alg2[i][j][4]*0.05

            if alg2[i][j][10] == 'supplier1':
                alg2[i][j].append(P2P_Bill)
                alg2[i][j].append(P2P_Reward)
                supplier_in1 = P2P_Bill
                supplier_out1 = P2P_Reward
                supplier_bal1 += supplier_in1 - supplier_out1
            if alg2[i][j][10] == 'supplier2':
                alg2[i][j].append(P2P_Bill)
                alg2[i][j].append(P2P_Reward)
                supplier_in2 = P2P_Bill
                supplier_out2 = P2P_Reward
                supplier_bal2 += supplier_in2 - supplier_out2
            if alg2[i][j][10] == 'supplier3':
                alg2[i][j].append(P2P_Bill)
                alg2[i][j].append(P2P_Reward)
                supplier_in3 = P2P_Bill
                supplier_out3 = P2P_Reward
                supplier_bal3 += supplier_in3 - supplier_out3
            if alg2[i][j][10] == 'supplier4':
                alg2[i][j].append(P2P_Bill)
                alg2[i][j].append(P2P_Reward)
                supplier_in4 = P2P_Bill
                supplier_out4 = P2P_Reward
                supplier_bal4 += supplier_in4 - supplier_out4
            if alg2[i][j][10] == 'supplier5':
                alg2[i][j].append(P2P_Bill)
                alg2[i][j].append(P2P_Reward)
                supplier_in5 = P2P_Bill
                supplier_out5 = P2P_Reward
                supplier_bal5 += supplier_in5 - supplier_out5

    supplier_bal_alg2.append([supplier_bal1,supplier_bal2,supplier_bal3,
                               supplier_bal4,supplier_bal5])

In [7]:
stat=alg2[24]
a = pd.DataFrame(stat, columns=['Client ID',
                                      'Consumption data',
                                      'PV data',
                                      'WT',
                                      'Consumption/generation profile',
                                      'Role',
                                      'Prediction algorithm',
                                      'Predicted volume',
                                      'Price offered',
                                      'Bid',
                                      'supplier',
                                      'retail price',
                                'alg1 bills', 'alg1 rewards',
                                'alg2 bills','alg2 rewards']
                 , index=[x[0] for x in stat])
a.T.sort_index(axis=1)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
Client ID,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
Consumption data,0.12,0.05,0.06,0.08,0.07,0.08,0.08,0.04,0.05,0.13,...,0.02,0.05,0.09,0.04,0.11,0.07,0.07,0.11,0.04,0.12
PV data,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Consumption/generation profile,0.12,0.05,0.06,0.08,0.07,0.08,0.08,0.04,0.05,0.13,...,0.02,0.05,0.09,0.04,0.11,0.07,0.07,0.11,0.04,0.12
Role,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,...,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer
Prediction algorithm,good,good,bad,good,good,bad,good,good,ok,good,...,good,good,good,good,good,good,perfect,good,ok,bad
Predicted volume,0.12,0.05,0.01,0.08,0.06,0.14,0.09,0.04,0.04,0.12,...,0.02,0.05,0.1,0.04,0.11,0.07,0.07,0.12,0.05,0.06
Price offered,0.16,0.08,0.05,0.13,0.1,0.18,0.14,0.07,0.07,0.16,...,0.06,0.1,0.15,0.08,0.16,0.12,0.12,0.17,0.08,0.11
Bid,rejected,rejected,rejected,rejected,rejected,accepted,rejected,rejected,rejected,rejected,...,rejected,rejected,rejected,rejected,rejected,rejected,rejected,rejected,rejected,rejected


# Algorithm 3: Billing Model with Social Cost Split


In [8]:
# alg3

alg3 = []
rent1=[]
rent2=[]
buyer_accepted=[]
seller_accepted=[]
supplier_bal_alg3 = []
alg3 = all_list


for i in range(len(alg3)):
    supplier_in1,supplier_in2,supplier_in3,supplier_in4,supplier_in5=0,0,0,0,0
    supplier_out1,supplier_out2,supplier_out3,supplier_out4,supplier_out5=0,0,0,0,0
    supplier_bal1,supplier_bal2,supplier_bal3,supplier_bal4,supplier_bal5=0,0,0,0,0
    supplier_out,supplier_in=0,0

    for j in range(len(alg3[i])):
        P2P_Bill=0
        P2P_Reward=0
        supplier_out,supplier_in=0,0
        supplier_in1,supplier_in2,supplier_in3,supplier_in4,supplier_in5=0,0,0,0,0
        supplier_out1,supplier_out2,supplier_out3,supplier_out4,supplier_out5=0,0,0,0,0

        if alg3[i][j][9] == 'accepted':
            if alg3[i][j][5] == 'buyer':
                rent1.append(j)
            if alg3[i][j][5] == 'seller':
                rent2.append(j)

        if  alg3[i][j][9] == 'rejected':
            if alg3[i][j][5] == 'buyer':
                P2P_Bill = alg3[i][j][4]*alg3[i][j][11]
            else:
                P2P_Reward = alg3[i][j][4]*0.05
            alg3[i][j].append(P2P_Bill)
            alg3[i][j].append(P2P_Reward)
            if alg3[i][j][10] == 'supplier1':
                supplier_in1 = P2P_Bill
                supplier_out1 = P2P_Reward
                supplier_bal1 += supplier_in1 - supplier_out1
            if alg3[i][j][10] == 'supplier2':
                supplier_in2 = P2P_Bill
                supplier_out2 = P2P_Reward
                supplier_bal2 += supplier_in2 - supplier_out2
            if alg3[i][j][10] == 'supplier3':
                supplier_in3 = P2P_Bill
                supplier_out3 = P2P_Reward
                supplier_bal3 += supplier_in3 - supplier_out3
            if alg3[i][j][10] == 'supplier4':
                supplier_in4 = P2P_Bill
                supplier_out4 = P2P_Reward
                supplier_bal4 += supplier_in4 - supplier_out4
            if alg3[i][j][10] == 'supplier5':
                supplier_in5 = P2P_Bill
                supplier_out5 = P2P_Reward
                supplier_bal5 += supplier_in5 - supplier_out5
    buyer_accepted.append(rent1)
    seller_accepted.append(rent2)
    rent1=[]
    rent2=[]
    supplier_bal_alg3.append([supplier_bal1,supplier_bal2,supplier_bal3,
                               supplier_bal4,supplier_bal5])

TDD=[]
TSD=[]
TD_rent=0
alg3_bill=0
alg3_reward=0

Tc_temp1 = 0
Tc_temp2 = 0
P2Pc_temp1 = 0
P2Pc_temp2 = 0
Tc_over = []
Tc_under = []
P2Pc_under = []
P2Pc_over=[]

for i in range(len(buyer_accepted)):
    for j in buyer_accepted[i]:
        TD_rent+=alg3[i][j][4]-alg3[i][j][7]

        if alg3[i][j][4]-alg3[i][j][7] > 0:
            Tc_temp1 += alg3[i][j][4]-alg3[i][j][7]
            P2Pc_temp2 += 1
        if alg3[i][j][4]-alg3[i][j][7] < 0:
            P2Pc_temp1 += 1
            Tc_temp2 += -(alg3[i][j][4]-alg3[i][j][7])

    TDD.append(TD_rent)
    Tc_over.append(Tc_temp1)
    Tc_under.append(Tc_temp2)
    P2Pc_over.append(P2Pc_temp2)
    P2Pc_under.append(P2Pc_temp1)
    Tc_temp1 = 0
    Tc_temp2 = 0
    P2Pc_temp1 = 0
    P2Pc_temp2 = 0
    TD_rent=0

for i in range(len(buyer_accepted)):
    if TDD[i] == 0:
        for j in buyer_accepted[i]:
            alg3_bill=alg3[i][j][4]*alg3[i][j][8]
            alg3[i][j].append(alg3_bill)
            alg3[i][j].append(0)
        #supplier_in1,supplier_in2,supplier_in3,supplier_in4,supplier_in5=0,0,0,0,0

    if TDD[i] < 0:
        for j in buyer_accepted[i]:
            if alg3[i][j][4]-alg3[i][j][7] == 0 or alg3[i][j][4]-alg3[i][j][7] > 0:
                alg3_bill=alg3[i][j][4]*alg3[i][j][8]
                alg3[i][j].append(alg3_bill)
                alg3[i][j].append(0)
                # supplier_in1,supplier_in2,supplier_in3,supplier_in4,supplier_in5=0,0,0,0,0

            if alg3[i][j][4]-alg3[i][j][7] < 0:
                alg3_bill = alg3[i][j][7]*alg3[i][j][8] + ((alg3[i][j][4]-alg3[i][j][7])+Tc_over[i]/P2Pc_under[i])*0.05
                alg3[i][j].append(alg3_bill)
                alg3[i][j].append(0)
                if alg3[i][j][10]=='supplier1':
                    supplier_bal_alg3[i][0]+=((alg3[i][j][4]-alg3[i][j][7])+Tc_over[i]/P2Pc_under[i])*0.05
                if alg3[i][j][10]=='supplier2':
                    supplier_bal_alg3[i][1]+=((alg3[i][j][4]-alg3[i][j][7])+Tc_over[i]/P2Pc_under[i])*0.05
                if alg3[i][j][10]=='supplier3':
                    supplier_bal_alg3[i][2]+=((alg3[i][j][4]-alg3[i][j][7])+Tc_over[i]/P2Pc_under[i])*0.05
                if alg3[i][j][10]=='supplier4':
                    supplier_bal_alg3[i][3]+=((alg3[i][j][4]-alg3[i][j][7])+Tc_over[i]/P2Pc_under[i])*0.05
                if alg3[i][j][10]=='supplier5':
                    supplier_bal_alg3[i][4]+=((alg3[i][j][4]-alg3[i][j][7])+Tc_over[i]/P2Pc_under[i])*0.05

    if TDD[i] > 0:
        for j in buyer_accepted[i]:
            if alg3[i][j][4]-alg3[i][j][7] == 0 or alg3[i][j][4]-alg3[i][j][7] < 0:
                alg3_bill=alg3[i][j][4]*alg3[i][j][8]
                alg3[i][j].append(alg3_bill)
                alg3[i][j].append(0)

            if alg3[i][j][4]-alg3[i][j][7] > 0:
                alg3_bill = (alg3[i][j][7]+Tc_under[i]/P2Pc_over[i])*alg3[i][j][8] + ((alg3[i][j][4]-alg3[i][j][7])-Tc_under[i]/P2Pc_over[i])*alg3[i][j][11]
                alg3[i][j].append(alg3_bill)
                alg3[i][j].append(0)
                if alg3[i][j][10]=='supplier1':
                    supplier_bal_alg3[i][0]+=((alg3[i][j][4]-alg3[i][j][7])-Tc_under[i]/P2Pc_over[i])*alg3[i][j][11]
                if alg3[i][j][10]=='supplier2':
                    supplier_bal_alg3[i][1]+=((alg3[i][j][4]-alg3[i][j][7])-Tc_under[i]/P2Pc_over[i])*alg3[i][j][11]
                if alg3[i][j][10]=='supplier3':
                    supplier_bal_alg3[i][2]+=((alg3[i][j][4]-alg3[i][j][7])-Tc_under[i]/P2Pc_over[i])*alg3[i][j][11]
                if alg3[i][j][10]=='supplier4':
                    supplier_bal_alg3[i][3]+=((alg3[i][j][4]-alg3[i][j][7])-Tc_under[i]/P2Pc_over[i])*alg3[i][j][11]
                if alg3[i][j][10]=='supplier5':
                    supplier_bal_alg3[i][4]+=((alg3[i][j][4]-alg3[i][j][7])-Tc_under[i]/P2Pc_over[i])*alg3[i][j][11]

Tp_over = []
Tp_under = []
P2Pp_under = []
P2Pp_over=[]
#p2pp
for i in range(len(seller_accepted)):
    for j in seller_accepted[i]:
        TD_rent+=alg3[i][j][4]-alg3[i][j][7]

        if alg3[i][j][4]-alg3[i][j][7] > 0:
            Tc_temp1 += alg3[i][j][4]-alg3[i][j][7]
            P2Pc_temp2 += 1
        if alg3[i][j][4]-alg3[i][j][7] < 0:
            P2Pc_temp1 += 1
            Tc_temp2 += -(alg3[i][j][4]-alg3[i][j][7])

    TSD.append(TD_rent)
    Tp_over.append(Tc_temp1)
    Tp_under.append(Tc_temp2)
    P2Pp_over.append(P2Pc_temp2)
    P2Pp_under.append(P2Pc_temp1)
    Tc_temp1 = 0
    Tc_temp2 = 0
    P2Pc_temp1 = 0
    P2Pc_temp2 = 0
    TD_rent=0

for i in range(len(seller_accepted)):
    if TSD[i] == 0:
        for j in seller_accepted[i]:
            alg3_reward = alg3[i][j][4]*alg3[i][j][8]
            alg3[i][j].append(0)
            alg3[i][j].append(alg3_reward)

    if TSD[i] < 0:
        for j in seller_accepted[i]:
            if alg3[i][j][4]-alg3[i][j][7] == 0 or alg3[i][j][4]-alg3[i][j][7] > 0:
                alg3_reward=alg3[i][j][4]*alg3[i][j][8]
                alg3[i][j].append(0)
                alg3[i][j].append(alg3_reward)

            if alg3[i][j][4]-alg3[i][j][7] < 0:
                alg3_reward = alg3[i][j][7]*alg3[i][j][8] + ((alg3[i][j][4]-alg3[i][j][7])+Tp_over[i]/P2Pp_under[i])*alg3[i][j][11]
                alg3[i][j].append(0)
                alg3[i][j].append(alg3_reward)
                if alg3[i][j][10]=='supplier1':
                    supplier_bal_alg3[i][0]-=((alg3[i][j][4]-alg3[i][j][7])+Tp_over[i]/P2Pp_under[i])*alg3[i][j][11]
                if alg3[i][j][10]=='supplier2':
                    supplier_bal_alg3[i][1]-=((alg3[i][j][4]-alg3[i][j][7])+Tp_over[i]/P2Pp_under[i])*alg3[i][j][11]
                if alg3[i][j][10]=='supplier3':
                    supplier_bal_alg3[i][2]-=((alg3[i][j][4]-alg3[i][j][7])+Tp_over[i]/P2Pp_under[i])*alg3[i][j][11]
                if alg3[i][j][10]=='supplier4':
                    supplier_bal_alg3[i][3]-=((alg3[i][j][4]-alg3[i][j][7])+Tp_over[i]/P2Pp_under[i])*alg3[i][j][11]
                if alg3[i][j][10]=='supplier5':
                    supplier_bal_alg3[i][4]-=((alg3[i][j][4]-alg3[i][j][7])+Tp_over[i]/P2Pp_under[i])*alg3[i][j][11]

    if TSD[i] > 0:
        for j in seller_accepted[i]:
            if alg3[i][j][4]-alg3[i][j][7] == 0 or alg3[i][j][4]-alg3[i][j][7] < 0:
                alg3_reward=alg3[i][j][4]*alg3[i][j][8]
                alg3[i][j].append(0)
                alg3[i][j].append(alg3_reward)

            if alg3[i][j][4]-alg3[i][j][7] > 0:
                alg3_reward = (alg3[i][j][7]+Tp_under[i]/P2Pp_over[i])*alg3[i][j][8] + ((alg3[i][j][4]-alg3[i][j][7])-Tp_under[i]/P2Pp_over[i])*0.05
                alg3[i][j].append(0)
                alg3[i][j].append(alg3_reward)
                if alg3[i][j][10]=='supplier1':
                    supplier_bal_alg3[i][0]-=((alg3[i][j][4]-alg3[i][j][7])-Tp_under[i]/P2Pp_over[i])*0.05
                if alg3[i][j][10]=='supplier2':
                    supplier_bal_alg3[i][1]-=((alg3[i][j][4]-alg3[i][j][7])-Tp_under[i]/P2Pp_over[i])*0.05
                if alg3[i][j][10]=='supplier3':
                    supplier_bal_alg3[i][2]-=((alg3[i][j][4]-alg3[i][j][7])-Tp_under[i]/P2Pp_over[i])*0.05
                if alg3[i][j][10]=='supplier4':
                    supplier_bal_alg3[i][3]-=((alg3[i][j][4]-alg3[i][j][7])-Tp_under[i]/P2Pp_over[i])*0.05
                if alg3[i][j][10]=='supplier5':
                    supplier_bal_alg3[i][4]-=((alg3[i][j][4]-alg3[i][j][7])-Tp_under[i]/P2Pp_over[i])*0.05

In [9]:
stat=alg3[10]
a = pd.DataFrame(stat,columns=['Client ID',
                                      'Consumption data',
                                      'PV data',
                                      'WT',
                                      'Consumption/generation profile',
                                      'Role',
                                      'Prediction algorithm',
                                      'Predicted volume',
                                      'Price offered',
                                      'Bid',
                                      'supplier',
                                      'retail price',
                                'alg1 bills', 'alg1 rewards',
                                'alg2 bills','alg2 rewards',
                              'alg3 bills','alg3 rewards']
                 , index=[x[0] for x in stat])
a.T.sort_index(axis=1)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
Client ID,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
Consumption data,0.05,0.05,0.07,0.75,1.16,0.57,0.09,0.2,0.05,0.36,...,0.46,0.05,0.77,0.06,0.12,1.2,2.64,0.94,0.18,0.1
PV data,0.0,1.35,0.0,0.0,0.0,0.0,0.0,1.89,1.28,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.88,0.0
WT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Consumption/generation profile,0.05,1.3,0.07,0.75,1.16,0.57,0.09,1.69,1.23,0.36,...,0.46,0.05,0.77,0.06,0.12,1.2,2.64,0.94,0.7,0.1
Role,buyer,seller,buyer,buyer,buyer,buyer,buyer,seller,seller,buyer,...,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,seller,buyer
Prediction algorithm,bad,ok,perfect,good,ok,bad,good,good,good,good,...,good,good,perfect,good,good,good,good,good,good,perfect
Predicted volume,0.02,1.12,0.07,0.76,1.29,0.04,0.09,1.71,1.26,0.33,...,0.43,0.05,0.77,0.06,0.11,1.13,2.73,0.9,0.68,0.1
Price offered,0.05,0.13,0.09,0.17,0.18,0.05,0.11,0.09,0.11,0.15,...,0.15,0.08,0.17,0.08,0.12,0.18,0.19,0.17,0.17,0.12
Bid,rejected,accepted,rejected,rejected,accepted,rejected,rejected,accepted,accepted,rejected,...,rejected,rejected,rejected,rejected,rejected,accepted,accepted,rejected,rejected,rejected


# Algorithm 4: Billing Model with Universal Cost Split


In [10]:
#alg4

alg4=[]
alg4=all_list
TD_rent1=0
TD_rent2=0
rent1=0
rent2=0
rent=[]

TD=[]
TD_accepted=[]
P2Pc_n=[]
P2Pp_n=[]
supplier_bal_alg4 = []

for i in range(len(alg4)):
    supplier_in1,supplier_in2,supplier_in3,supplier_in4,supplier_in5=0,0,0,0,0
    supplier_out1,supplier_out2,supplier_out3,supplier_out4,supplier_out5=0,0,0,0,0
    supplier_bal1,supplier_bal2,supplier_bal3,supplier_bal4,supplier_bal5=0,0,0,0,0
    supplier_out,supplier_in=0,0

    for j in range(len(alg4[i])):
        P2P_Bill=0
        P2P_Reward=0
        supplier_out,supplier_in=0,0
        supplier_in1,supplier_in2,supplier_in3,supplier_in4,supplier_in5=0,0,0,0,0
        supplier_out1,supplier_out2,supplier_out3,supplier_out4,supplier_out5=0,0,0,0,0

        if alg4[i][j][9] == 'accepted':

            rent.append(j)
            if alg4[i][j][5] == 'buyer':

                TD_rent1 += alg4[i][j][4]-alg4[i][j][7]
                if alg4[i][j][4]-alg4[i][j][7] > 0:
                    rent1+=1
            if alg4[i][j][5] == 'seller':

                TD_rent2 += alg4[i][j][4]-alg4[i][j][7]
                if alg4[i][j][4]-alg4[i][j][7] > 0:
                    rent2+=1

        if alg4[i][j][9] == 'rejected':
            if alg3[i][j][5] == 'buyer':
                P2P_Bill = alg4[i][j][4]*alg4[i][j][11]
            else:
                P2P_Reward = alg4[i][j][4]*0.05
            alg4[i][j].append(P2P_Bill)
            alg4[i][j].append(P2P_Reward)
            if alg4[i][j][10] == 'supplier1':
                supplier_in1 = P2P_Bill
                supplier_out1 = P2P_Reward
                supplier_bal1 += supplier_in1 - supplier_out1
            if alg4[i][j][10] == 'supplier2':
                supplier_in2 = P2P_Bill
                supplier_out2 = P2P_Reward
                supplier_bal2 += supplier_in2 - supplier_out2
            if alg4[i][j][10] == 'supplier3':
                supplier_in3 = P2P_Bill
                supplier_out3 = P2P_Reward
                supplier_bal3 += supplier_in3 - supplier_out3
            if alg4[i][j][10] == 'supplier4':
                supplier_in4 = P2P_Bill
                supplier_out4 = P2P_Reward
                supplier_bal4 += supplier_in4 - supplier_out4
            if alg4[i][j][10] == 'supplier5':
                supplier_in5 = P2P_Bill
                supplier_out5 = P2P_Reward
                supplier_bal5 += supplier_in5 - supplier_out5


    TD.append(TD_rent2-TD_rent1)
    TD_accepted.append(rent)
    P2Pc_n.append(rent1)
    P2Pp_n.append(rent2)
    rent=[]
    TD_rent=0
    supplier_bal_alg4.append([supplier_bal1,supplier_bal2,supplier_bal3,
                               supplier_bal4,supplier_bal5])

alg4_bill=0
alg4_reward=0

for i in range(len(TD_accepted)):
    if TD[i] == 0:
        for j in TD_accepted[i]:
            if alg4[i][j][5] == 'buyer':
                alg4_bill = alg4[i][j][4]*alg4[i][j][8]
                alg4[i][j].append(alg4_bill)
                alg4[i][j].append(0)
            if alg4[i][j][5] == 'seller':
                alg4_reward = alg4[i][j][4]*alg4[i][j][8]
                alg4[i][j].append(0)
                alg4[i][j].append(alg4_reward)

    if TD[i] < 0:
        for j in TD_accepted[i]:
            if alg4[i][j][5] == 'seller':
                alg4_reward = alg4[i][j][4]*alg4[i][j][8]
                alg4[i][j].append(0)
                alg4[i][j].append(alg4_reward)
            if alg4[i][j][5] == 'buyer':
                if alg4[i][j][4]-alg4[i][j][7] == 0 or alg4[i][j][4]-alg4[i][j][7] < 0:
                    alg4_bill = alg4[i][j][4]*alg4[i][j][8]


                if alg4[i][j][4]-alg4[i][j][7] > 0:
                    alg4_bill = (alg4[i][j][7]+TD[i]/P2Pc_n[i])*alg4[i][j][8]+(-TD[i]/P2Pc_n[i])*alg4[i][j][11]

                    if alg4[i][j][10]=='supplier1':
                        supplier_bal_alg4[i][0]+=(-TD[i]/P2Pc_n[i])*alg4[i][j][11]
                    if alg4[i][j][10]=='supplier2':
                        supplier_bal_alg4[i][1]+=(-TD[i]/P2Pc_n[i])*alg4[i][j][11]
                    if alg4[i][j][10]=='supplier3':
                        supplier_bal_alg4[i][2]+=(-TD[i]/P2Pc_n[i])*alg4[i][j][11]
                    if alg4[i][j][10]=='supplier4':
                        supplier_bal_alg4[i][3]+=(-TD[i]/P2Pc_n[i])*alg4[i][j][11]
                    if alg4[i][j][10]=='supplier5':
                        supplier_bal_alg4[i][4]+=(-TD[i]/P2Pc_n[i])*alg4[i][j][11]
                alg4[i][j].append(alg4_bill)
                alg4[i][j].append(0)

    if TD[i] > 0:
        for j in TD_accepted[i]:
            if alg4[i][j][5] == 'buyer':
                alg4_bill = alg4[i][j][4]*alg4[i][j][8]
                alg4[i][j].append(alg4_bill)
                alg4[i][j].append(0)
            if alg4[i][j][5] == 'seller':
                if alg4[i][j][4]-alg4[i][j][7] == 0 or alg4[i][j][4]-alg4[i][j][7] < 0:
                    alg4_reward = alg4[i][j][4]*alg4[i][j][8]

                if alg4[i][j][4]-alg4[i][j][7] > 0:
                    alg4_reward = (alg4[i][j][7]-TD[i]/P2Pp_n[i])*alg4[i][j][8]+(TD[i]/P2Pp_n[i])*0.05

                    if alg4[i][j][10]=='supplier1':
                        supplier_bal_alg4[i][0]-=(TD[i]/P2Pp_n[i])*0.05
                    if alg4[i][j][10]=='supplier2':
                        supplier_bal_alg4[i][1]-=(TD[i]/P2Pp_n[i])*0.05
                    if alg4[i][j][10]=='supplier3':
                        supplier_bal_alg4[i][2]-=(TD[i]/P2Pp_n[i])*0.05
                    if alg4[i][j][10]=='supplier4':
                        supplier_bal_alg4[i][3]-=(TD[i]/P2Pp_n[i])*0.05
                    if alg4[i][j][10]=='supplier5':
                        supplier_bal_alg4[i][4]-=(TD[i]/P2Pp_n[i])*0.05
                alg4[i][j].append(0)
                alg4[i][j].append(alg4_reward)

In [15]:
stat=alg4[11]
a = pd.DataFrame(stat,columns=['Client ID',
                                      'Consumption data',
                                      'PV data',
                                      'WT',
                                      'Consumption/generation profile',
                                      'Role',
                                      'Prediction algorithm',
                                      'Predicted volume',
                                      'Price offered',
                                      'Bid',
                                      'supplier',
                                      'retail price',
                                'alg1 bills', 'alg1 rewards',
                                'alg2 bills','alg2 rewards',
                              'alg3 bills','alg3 rewards',
                              'alg4 bills','alg4 rewards']
                 , index=[x[0] for x in stat])
a.T.sort_index(axis=1)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
Client ID,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
Consumption data,0.05,0.05,1.93,0.4,1.01,0.09,0.09,0.15,0.04,0.15,...,2.76,0.05,0.45,0.04,0.1,1.04,1.28,0.91,0.14,0.09
PV data,0.0,1.95,0.0,0.0,0.0,0.0,0.0,2.11,1.73,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.25,0.0
WT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Consumption/generation profile,0.05,1.9,1.93,0.4,1.01,0.09,0.09,1.96,1.69,0.15,...,2.76,0.05,0.45,0.04,0.1,1.04,1.28,0.91,1.11,0.09
Role,buyer,seller,buyer,buyer,buyer,buyer,buyer,seller,seller,buyer,...,buyer,buyer,buyer,buyer,buyer,buyer,buyer,buyer,seller,buyer
Prediction algorithm,good,perfect,good,good,good,ok,good,good,ok,good,...,ok,bad,good,ok,perfect,perfect,good,good,ok,good
Predicted volume,0.05,1.9,1.85,0.41,0.93,0.08,0.08,2.07,1.5,0.14,...,2.43,0.07,0.49,0.03,0.1,1.04,1.35,0.87,0.97,0.1
Price offered,0.06,0.13,0.18,0.16,0.17,0.08,0.08,0.12,0.15,0.13,...,0.19,0.08,0.16,0.05,0.12,0.17,0.18,0.16,0.17,0.12
Bid,rejected,accepted,accepted,rejected,rejected,rejected,rejected,accepted,accepted,rejected,...,accepted,rejected,rejected,rejected,rejected,rejected,accepted,rejected,rejected,rejected
